## dados tabulares

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.tabular.models import *

[`tabular`](/tabular.html#tabular) contém todas as classes necessárias para lidar com dados tabulares, através de dois módulos:
- [`tabular.transform`](/tabular.transform.html#tabular.transform): define a classe [`TabularTransform`](/tabular.transform.html#TabularTransform) para ajudar com pré-processamento;
- [`tabular.data`](/tabular.data.html#tabular.data): define o [`TabularDataset`](/tabular.data.html#TabularDataset) que lida com esses dados, bem como os métodos para obter rapidamente uma [`TabularDataBunch`](/tabular.data.html#TabularDataBunch).
Para criar um modelo, você vai precisar usar [`models.tabular`](/tabular.html#tabular). Ver abaixo para um exemplo de ponta a ponta usando todos estes módulos.

## Pré-processamento de dados tabulares

Primeiro, vamos importar tudo o que precisamos para a aplicação tabular.

In [ ]:
from fastai.tabular import * 

dados tabulares geralmente vem na forma de um arquivo delimitado (como .csv) contendo variáveis ​​de diferentes tipos: text / categoria, números e talvez alguns valores em falta. O exemplo vamos trabalhar com nesta seção é uma amostra do [adult dataset](https://archive.ics.uci.edu/ml/datasets/adult) que tem algumas informações do censo sobre os indivíduos. Vamos usá-lo para treinar um modelo para prever se salário é maior que \ $ 50k ou não.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/adult_sample')

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


Aqui todas as informações que irá formar a nossa entrada está nas 14 primeiras colunas, ea variável dependente é a última coluna. Vamos dividir a nossa entrada entre dois tipos de variáveis: categóricas e contínuas.
- As variáveis ​​categóricas será substituído por uma categoria - uma identificação única que identifica-los - antes que eles são passados ​​através de uma camada de incorporação.
- As variáveis ​​contínuas será normalizado e, em seguida, introduzida directamente no modelo.
Outra coisa que precisa lidar são os valores em falta: o nosso modelo não vai gostar de receber NaNs por isso, devemos removê-los de uma forma inteligente. Tudo isso de pré-processamento é feito por objetos [`TabularTransform`](/tabular.transform.html#TabularTransform) e [`TabularDataset`](/tabular.data.html#TabularDataset).
Podemos definir um grupo de transformações que serão aplicadas às nossas variáveis. Aqui nós transformar todas as variáveis ​​categóricas em categorias. Nós também substituir os valores em falta para variáveis ​​contínuas pelo valor da coluna mediana e normalizar os.

In [ ]:
procs = [FillMissing, Categorify, Normalize]

Para dividir nossos dados para treinamento e validação conjuntos, usamos índices válidos

In [ ]:
valid_idx = range(len(df)-2000, len(df))

Então vamos dividir manualmente nossas variáveis ​​em variáveis ​​categóricas e contínuas (podemos ignorar a variável dependente nesta fase). fastai assumirá todas as variáveis ​​que não são dependentes ou categóricas são contínuas, a menos que explicitamente passar uma lista para o parâmetro `cont_names` ao construir nossa [`DataBunch`](/basic_data.html#DataBunch).

In [ ]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

Agora estamos prontos para passar essas informações para [`TabularDataBunch.from_df`](/tabular.data.html#TabularDataBunch.from_df) para criar o [`DataBunch`](/basic_data.html#DataBunch) que vamos usar para treinamento.

In [ ]:
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

['capital-gain', 'fnlwgt', 'hours-per-week', 'capital-loss', 'education-num', 'age']


Podemos pegar uma mini-série de dados e vejam (note que [`to_np`](/torch_core.html#to_np) aqui converte de tensor pytorch para Numpy):

In [ ]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[ 5  8  3 13  1  5  2 40  1]
 [ 5 12  3  5  1  3  2 40  1]
 [ 1  2  3  1  1  5  2 40  1]
 [ 5 12  3  4  1  5  2 40  1]
 [ 7 12  3  5  1  5  2 40  1]]
[[ 0.886658 -1.382477 -0.035789 -0.216787  0.753904  0.393667]
 [-0.145922 -0.367974  1.176085  4.469497 -0.421569  0.833029]
 [-0.145922 -1.454197 -2.621122 -0.216787 -1.205218  2.07789 ]
 [-0.145922  0.135373 -0.035789 -0.216787 -0.421569  0.173985]
 [-0.145922 -0.224013 -0.035789 -0.216787 -0.421569  2.151118]]
[1 1 0 0 1]


Depois de ter sido processado em [`TabularDataset`](/tabular.data.html#TabularDataset), as variáveis ​​categóricas são substituídos por ids e as variáveis ​​contínuas são normalizados. Os códigos correspondentes às variáveis ​​categóricas são todos juntos, como são todas as variáveis ​​contínuas.

## A definição de um modelo

Uma vez que temos nossos dados prontos em um [`DataBunch`](/basic_data.html#DataBunch), só precisamos criar um modelo para, em seguida, definir uma [`Learner`](/basic_train.html#Learner) e começar a treinar. A biblioteca fastai tem uma [`TabularModel`](/tabular.models.html#TabularModel) flexível e poderosa em [`models.tabular`](/tabular.html#tabular). Para usar essa função, só precisamos especificar os tamanhos de incorporação para cada uma das nossas variáveis ​​categóricas.

In [ ]:
learn = tabular_learner(data, layers=[200,100], emb_szs={'native-country': 10}, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.321540,0.319863,0.844000,00:04


Como de costume, podemos usar o método [`Learner.predict`](/basic_train.html#Learner.predict) para obter previsões. Neste caso, é preciso passar a linha de uma trama de dados que tem os mesmos nomes de variáveis ​​categóricas e contínuas como a nossa formação ou trama de dados de validação.

In [ ]:
learn.predict(df.iloc[0])

(Category >=50k, tensor(1), tensor([0.1864, 0.8136]))